In [5]:
from bertopic import BERTopic
import pandas as pd
import re
import spacy

In [29]:
df = pd.read_csv('../dataset/anfragen_oldenburg_detailed.csv')

# Pre-processing data

In [30]:
# lowercasing
df['Beschreibung'] = df['Beschreibung'].str.lower()

# removal of punctuation and digits
def clean_entries(entry):
    entry = re.sub(r'[\d]', ' ', entry) # remove digits
    entry = re.sub(r'[^\w\s]', ' ', entry) # remove non-word characters
    return entry
df['Beschreibung'] = df['Beschreibung'].apply(clean_entries)

# removal of stopwords (e.g., der, die, das, ein)
nlp = spacy.load("de_core_news_sm")
def remove_stopwords(entry):
    doc = nlp(entry)
    return " ".join([token.text for token in doc if not token.is_stop and not token.is_punct and not token.is_space])
df['Beschreibung'] = df['Beschreibung'].apply(remove_stopwords)

# lemmatization (e.g., reducing gegangen to gehen)
def lemmatize(entry):
    doc = nlp(entry)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space])
df['Beschreibung'] = df['Beschreibung'].apply(lemmatize)

# remove street names
def remove_street_names(entry):
    # remove words ending with straße, strasse, allee, gasse, or weg (except for einweg, which could be trash)
    pattern = r'\b(?!einweg\b)\w*(straße|strasse|str|weg|allee|gasse|nummer|nr)\b'
    return re.sub(pattern, ' ', entry, flags=re.IGNORECASE)
df['Beschreibung'] = df['Beschreibung'].apply(remove_street_names)

In [31]:
display(df['Beschreibung'].head())

0    Famila   liegen rechts Müll Gebüsch bitte entf...
1    Ecke nadorst   lindenhofsgaren diverser Abfall...
2      Gang     Höhe   erneut illegal Abfall entsorgen
3    Containerstellplatz Ecke nedderend diverser Ha...
4                          Gefrier Kühlschrank ablegen
Name: Beschreibung, dtype: object

# Topic model

In [32]:
docs = df['Beschreibung'].dropna()

In [33]:
topic_model = BERTopic(language="german", min_topic_size=5)
topics, probs = topic_model.fit_transform(docs)

In [34]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,235,-1_liegen_richtung_woche_höhe,"[liegen, richtung, woche, höhe, stehen, befind...",[pferdemarken gebüsch Parkplatz pferdemarken v...
1,0,49,0_sperrmüll_gebotszeich_drückampel_gulli,"[sperrmüll, gebotszeich, drückampel, gulli, fa...","[ unrat bermen Rinne Farbeimer, willi Trinne ..."
2,1,41,1_fußball_werbung_unterhaltungsfußball_gmbh,"[fußball, werbung, unterhaltungsfußball, gmbh,...",[Aufkleber Werbung Unterhaltungsfußball vfb Ol...
3,2,36,2_auto_fahrzeug_autofahrer_gefährlich,"[auto, fahrzeug, autofahrer, gefährlich, heste...",[Carl Friedrich Gauss Höhe Steh Fahrzeug M...
4,3,34,3_müll_mülltonnen_abfall_mülleimer,"[müll, mülltonnen, abfall, mülleimer, überfüll...","[Müll, Müll, Silvester mülltonnen brannen steh..."
5,4,32,4_bushaltestellen_bus_schwelle_busverkehr,"[bushaltestellen, bus, schwelle, busverkehr, s...","[bushaltestellen voll Müll, hinter Ressource Z..."
6,5,26,5_entfernen_gmbh_fußball_aufkleber,"[entfernen, gmbh, fußball, aufkleber, werbung,...",[Aufkleber Werbung Unterhaltungsfußball vfb Ol...
7,6,25,6_herr_rechts_spielplatz_gemeinsam,"[herr, rechts, spielplatz, gemeinsam, heckklap...",[Spielplatz Ludwig freese strn bümmersteden hä...
8,7,24,7_kennzeichen_stehen_abgemeldet_parkstreifen,"[kennzeichen, stehen, abgemeldet, parkstreifen...",[öffentlich Parkplatz stehen abgemeldet rot Fe...
9,8,23,8_regenwasser_wasser_laufen_entwässerung,"[regenwasser, wasser, laufen, entwässerung, ab...",[Höhe Grundschule sammeln Regen Wasser Mann ...
